In [15]:
import numpy as np
import pandas as pd
import json
import pickle

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [16]:
import sys, os
sys.path.append(os.path.join(os.path.abspath(''), '..', 'shared_libs'))
import data_transform

In [17]:
df_valid = pd.read_csv('data/data_valid.csv', dtype={"zipcode":str})
df_valid.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId
0,For sale,NaN,Single Family,6232 Wellington Pl,3 ba,"{'atAGlanceFacts': [{'factValue': '1994', 'fac...",NaN,Dayton,"[{'rating': ['4/10', '4/10', '3/10'], 'data': ...","1,820 sqft",45424,4 bd,OH,NaN,806638,NaN,NaN
1,For sale,NaN,Miscellaneous,121 Mustang,3.0,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",NaN,Irvine,"[{'rating': ['9/10', '9/10', '9/10'], 'data': ...","2,162",92602,4,CA,2,NaN,Yes,CV19257861
2,for sale,NaN,single-family home,5545 High Bank Rd,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Fort Worth,"[{'rating': ['3/10', '8/10', '5/10'], 'data': ...","2,367 sqft",76126,3 Beds,TX,2.0,NaN,yes,NaN
3,for sale,NaN,single-family home,294 SW Ridgecrest Dr,3 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Port Saint Lucie,"[{'rating': ['8/10', '5/10', '4/10', '3/10', '...","2,270 sqft",34953,3 Beds,FL,1.0,NaN,NaN,RX-10573870
4,Active,NaN,Single Family Home,2782 Bookmark Dr,5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Kissimmee,"[{'rating': [], 'data': {'Distance': [], 'Grad...",3014,34746,5,FL,2,NaN,NaN,T3210031


In [18]:
target = pd.read_csv('data/data_valid_target.csv')

In [19]:
df = data_transform.clear_data_base_line(
    df_valid, 
    '../shared_libs/data/default_values.pkl', 
    can_drop_rows=False, 
    force_rebuild_cached_data=False
)

In [20]:
cities_dict = data_transform.get_cities_dict('../shared_libs/data/cities_dict.pkl', force_read=True)
address_dict = data_transform.get_addresses_dict('../shared_libs/data/address_dict.pkl', force_read=True)
address_by_zip_dict = data_transform.get_address_by_zipcode_dict('../shared_libs/data/address_by_zip_dict.pkl', force_read=True)
cities_clusters_dict = data_transform.get_citiess_clusters_dict('../shared_libs/data/cities_clusters_dict.pkl', force_read=True)

In [21]:
df = data_transform.fix_incorrect_states_and_cities(
    df, 
    '../shared_libs/data/default_values.pkl', 
    cities_dict, 
    can_drop_rows=False
)

In [22]:
df = data_transform.add_city_features(
    df,
    '../shared_libs/data/default_values.pkl', 
    cities_dict, 
    address_dict,
    address_by_zip_dict,
    cities_clusters_dict,
    force_rebuild_cached_data=False
)

In [23]:
df = data_transform.add_population_features(
    df, 
    '../shared_libs/data/default_values.pkl', 
    '../shared_libs/data/uscities.csv', 
    can_drop_rows=False,
    force_rebuild_cached_data=False
)

In [24]:
df = data_transform.encode_state_and_city(
    df, 
    '../shared_libs/data/default_values.pkl', 
    can_drop_rows=False, 
    force_rebuild_cached_data=False
)

In [25]:
df = data_transform.final_tune_pca_and_scale(
    df, 
    '../shared_libs/data/default_values.pkl', 
    force_rebuild_cached_data=False
)

In [26]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [27]:
y = target['target']
X = df

In [28]:
with open('../shared_libs/data/models/model_abr.pkl', 'rb') as f:
    model = pickle.load(f)

y_pred = model.predict(X)

print('MAPE:', mean_absolute_percentage_error(y, y_pred)*100)
print('RMSE:', mean_squared_error(y, y_pred)**0.5)

#MAPE: 29.371809225027334
#RMSE: 301300.2473079713

MAPE: 29.371809225027334
RMSE: 301300.2473079713
